In [1]:
import casadi as ca
import matplotlib.pyplot as plt
import numpy as np
from collections import namedtuple


from IPython.display import HTML, display
import matplotlib.animation as animation


In [2]:
# Constants
nq = 3 + 4
save_gifs = True
nv = 3
nu = 2 + 2 + 4
g = 9.81

# Physical parameters
m = 12.0  # kg (similar to mini cheetah)
L = 0.7  # body length (m)
inertia = 1 / 12 * m * (L**2)  # moment of inertia for rectangular body

leg_length = 0.3  # m

ForwardKinematicsResult = namedtuple(
    "ForwardKinematicsResult",
    [
        "com_x",
        "com_y",
        "front_shoulder_x",
        "front_shoulder_y",
        "front_knee_x",
        "front_knee_y",
        "front_ankle_x",
        "front_ankle_y",
        "rear_shoulder_x",
        "rear_shoulder_y",
        "rear_knee_x",
        "rear_knee_y",
        "rear_ankle_x",
        "rear_ankle_y",
    ],
)


def forward_kinematics(q):
    lib = ca
    if isinstance(q, np.ndarray):
        lib = np

    out = dict()
    out["com_x"] = q[0]
    out["com_y"] = q[1]

    phi = q[2]

    for multiplier, prefix in zip([1, -1], ["front", "rear"]):
        if prefix == "front":
            theta1 = q[3]
            theta2 = q[4]
        else:
            theta1 = q[5]
            theta2 = q[6]

        LL = multiplier * L
        shoulder_x = out["com_x"] + LL / 2 * lib.cos(phi)
        shoulder_y = out["com_y"] + LL / 2 * lib.sin(phi)

        knee_x = shoulder_x + leg_length * lib.cos(phi - lib.pi / 2 * 1 + theta1)
        knee_y = shoulder_y + leg_length * lib.sin(phi - lib.pi / 2 * 1 + theta1)

        ankle_x = knee_x + leg_length * lib.cos(phi - lib.pi / 2 * 1 + theta1 + theta2)
        ankle_y = knee_y + leg_length * lib.sin(phi - lib.pi / 2 * 1 + theta1 + theta2)

        out[f"{prefix}_shoulder_x"] = shoulder_x
        out[f"{prefix}_shoulder_y"] = shoulder_y
        out[f"{prefix}_knee_x"] = knee_x
        out[f"{prefix}_knee_y"] = knee_y
        out[f"{prefix}_ankle_x"] = ankle_x
        out[f"{prefix}_ankle_y"] = ankle_y

    return ForwardKinematicsResult(**out)


def euler_integrate(q, v, u, dt):
    x, y, phi = q[0], q[1], q[2]
    vx, vy, omega = v[0], v[1], v[2]
    theta1, theta2, theta3, theta4 = q[3], q[4], q[5], q[6]
    dtheta1, dtheta2, dtheta3, dtheta4 = u[4], u[5], u[6], u[7]
    F1_x, F1_y, F2_x, F2_y = u[0], u[1], u[2], u[3]

    # Integrate
    x_next = x + vx * dt
    y_next = y + vy * dt
    phi_next = phi + omega * dt

    theta1_next = theta1 + dtheta1 * dt
    theta2_next = theta2 + dtheta2 * dt
    theta3_next = theta3 + dtheta3 * dt
    theta4_next = theta4 + dtheta4 * dt

    ax = (F1_x + F2_x) / m
    ay = (F1_y + F2_y) / m - g  # gravity

    fk = forward_kinematics(q)

    # Compute position of the feet relative to the COM
    rfront_x = fk.front_ankle_x - fk.com_x
    rfront_y = fk.front_ankle_y - fk.com_y
    rrear_x = fk.rear_ankle_x - fk.com_x
    rrear_y = fk.rear_ankle_y - fk.com_y

    wrench = (F1_x * rfront_y - F1_y * rfront_x) + (F2_x * rrear_y - F2_y * rrear_x)
    eps = 1 / inertia * wrench

    vx_next = vx + ax * dt
    vy_next = vy + ay * dt
    omega_next = omega + eps * dt

    q_next = np.array([x_next, y_next, phi_next, theta1_next, theta2_next, theta3_next, theta4_next])
    v_next = np.array([vx_next, vy_next, omega_next])

    return q_next, v_next


class QuadrupedVisualizer:
    def __init__(self, L, leg_length):
        self.L = L
        self.leg_length = leg_length

        # Initialize the figure and axis
        self.fig, self.ax = plt.subplots()
        self.ax.set_aspect("equal")
        self.ax.set_xlim(-1, 2.2)
        self.ax.set_ylim(0.0, 2.5)

        # Initialize plot elements
        (self.com_plot,) = self.ax.plot([], [], "ko", markersize=10, label="COM")
        (self.shoulder_plots,) = self.ax.plot([], [], "bo", markersize=10, label="Shoulders")
        (self.knee_plots,) = self.ax.plot([], [], "ro", markersize=10, label="Knees")
        (self.ankle_plots,) = self.ax.plot([], [], "go", markersize=10, label="Ankles")
        (self.link_lines_front,) = self.ax.plot([], [], "r-", linewidth=2)
        (self.link_lines_rear,) = self.ax.plot([], [], "r-", linewidth=2)
        (self.force_lines_front,) = self.ax.plot([], [], "r-", linewidth=2)
        (self.force_lines_rear,) = self.ax.plot([], [], "r-", linewidth=2)
        (self.q_trajectory,) = self.ax.plot([], [], "k--", lw=1, alpha=0.5)

        # Draw ground
        self.ax.plot([-1, 0.2], [0.55, 0.55], "k-", linewidth=2)
        self.ax.plot([0.2, 0.2], [0.55, 0.75], "k-", linewidth=2)
        self.ax.plot([0.2, 2.3], [0.75, 0.75], "k-", linewidth=2)

    def set_data(self, q, u=None, q_trajectory=None):
        com_x = q[0]
        com_y = q[1]
        phi = q[2]
        theta1 = q[3]
        theta2 = q[4]
        theta3 = q[5]
        theta4 = q[6]

        # Compute positions
        shoulder_front_x = com_x + self.L / 2 * np.cos(phi)
        shoulder_front_y = com_y + self.L / 2 * np.sin(phi)

        shoulder_rear_x = com_x - self.L / 2 * np.cos(phi)
        shoulder_rear_y = com_y - self.L / 2 * np.sin(phi)

        knee_front_x = shoulder_front_x + self.leg_length * np.cos(phi - np.pi / 2 * 1 + theta1)
        knee_front_y = shoulder_front_y + self.leg_length * np.sin(phi - np.pi / 2 * 1 + theta1)

        knee_rear_x = shoulder_rear_x + self.leg_length * np.cos(phi - np.pi / 2 * 1 + theta3)
        knee_rear_y = shoulder_rear_y + self.leg_length * np.sin(phi - np.pi / 2 * 1 + theta3)

        ankle_front_x = knee_front_x + self.leg_length * np.cos(phi - np.pi / 2 * 1 + theta1 + theta2)
        ankle_front_y = knee_front_y + self.leg_length * np.sin(phi - np.pi / 2 * 1 + theta1 + theta2)

        ankle_rear_x = knee_rear_x + self.leg_length * np.cos(phi - np.pi / 2 * 1 + theta3 + theta4)
        ankle_rear_y = knee_rear_y + self.leg_length * np.sin(phi - np.pi / 2 * 1 + theta3 + theta4)

        # Update COM
        self.com_plot.set_data([com_x], [com_y])

        # Update shoulders
        self.shoulder_plots.set_data([shoulder_front_x, shoulder_rear_x], [shoulder_front_y, shoulder_rear_y])

        # Update knees
        self.knee_plots.set_data([knee_front_x, knee_rear_x], [knee_front_y, knee_rear_y])

        # Update ankles
        self.ankle_plots.set_data([ankle_front_x, ankle_rear_x], [ankle_front_y, ankle_rear_y])

        # Update links
        self.link_lines_front.set_data(
            [com_x, shoulder_front_x, knee_front_x, ankle_front_x],
            [com_y, shoulder_front_y, knee_front_y, ankle_front_y],
        )
        self.link_lines_rear.set_data(
            [com_x, shoulder_rear_x, knee_rear_x, ankle_rear_x],
            [com_y, shoulder_rear_y, knee_rear_y, ankle_rear_y],
        )

        if u is not None:
            F1_x, F1_y, F2_x, F2_y = u[:4] / 150
            self.force_lines_front.set_data(
                [ankle_front_x, ankle_front_x + F1_x], [ankle_front_y, ankle_front_y + F1_y]
            )
            self.force_lines_rear.set_data([ankle_rear_x, ankle_rear_x + F2_x], [ankle_rear_y, ankle_rear_y + F2_y])

        if q_trajectory is not None:
            self.q_trajectory.set_data(q_trajectory[0, :], q_trajectory[1, :])

        plt.draw()

In [ ]:
# Create state vector [x, y, phi, theta1-4]
q_initial = np.array([1.7, 1.2, 0, -1, 1.4, -1, +1.4])
v_initial = np.array([0, 0, 0])

q_final = np.array([-0.3, 1.0, 2 * np.pi, -1, 1.4, -1, +1.4])
v_final = np.array([0, 0, 0])


visualizer = QuadrupedVisualizer(L, leg_length)
visualizer.set_data(q_initial)
plt.show()

q = ca.SX.sym("q", nq)
v = ca.SX.sym("v", nv)
u = ca.SX.sym("u", nu)

dt = 0.02

q_next, v_next = euler_integrate(q, v, u, dt)

get_next_state = ca.Function("get_next_state", [q, v, u], [q_next, v_next])

opti = ca.Opti()

N = 100

Q = opti.variable(nq, N)
V = opti.variable(nv, N)
U = opti.variable(nu, N - 1)


opti.subject_to(Q[:, 0] == q_initial)
opti.subject_to(V[:, 0] == v_initial)

opti.subject_to(Q[:, -1] == q_final)
opti.subject_to(V[:, -1] == v_final)

# Both feet on the ground
PHASE1_START = 0
PHASE1_END = int(0.25 * N)

# One foot on the ground
PHASE2_START = PHASE1_END
PHASE2_END = int(0.35 * N)

# Both feet off the ground
PHASE3_START = PHASE2_END
PHASE3_END = int(0.65 * N)

# One foot on the ground
PHASE4_START = PHASE3_END
PHASE4_END = int(0.68 * N)

# Both feet off the ground
PHASE5_START = PHASE4_END
PHASE5_END = N

def add_friction_cone_constraint(opti, U, k, mu, leg):

    if leg == "front":
        index = [0, 1]
    elif leg == "rear":
        index = [2, 3]
    else:
        raise ValueError("Invalid leg")

    fx = U[index[0], k]
    fy = U[index[1], k]
    opti.subject_to(fx >= -mu * fy)
    opti.subject_to(fx <= mu * fy)

def add_zero_force_constraint(opti, U, k, leg):
    if leg == "front":
        index = [0, 1]
    elif leg == "rear":
        index = [2, 3]
    else:
        raise ValueError("Invalid leg")
    
    fx = U[index[0], k]
    fy = U[index[1], k]
    opti.subject_to(fx == 0)
    opti.subject_to(fy == 0)

def add_contact_constraint(opti, Q, k, q, leg):
    fk_current = forward_kinematics(q)
    fk_symbolic = forward_kinematics(Q[:, k])
    if leg == "front":
        current_front_leg_x, current_front_leg_y = fk_current.front_ankle_x, fk_current.front_ankle_y
        front_leg_x, front_leg_y = fk_symbolic.front_ankle_x, fk_symbolic.front_ankle_y

        opti.subject_to(front_leg_x == current_front_leg_x)
        opti.subject_to(front_leg_y == current_front_leg_y)
    elif leg == "rear":
        current_rear_leg_x, current_rear_leg_y = fk_current.rear_ankle_x, fk_current.rear_ankle_y
        rear_leg_x, rear_leg_y = fk_symbolic.rear_ankle_x, fk_symbolic.rear_ankle_y

        opti.subject_to(rear_leg_x == current_rear_leg_x)
        opti.subject_to(rear_leg_y == current_rear_leg_y)
    else:
        raise ValueError("Invalid leg")

mu = 0.99
for k in range(N - 1):
    q_next, v_next = get_next_state(Q[:, k], V[:, k], U[:, k])
    opti.subject_to(Q[:, k + 1] == q_next)
    opti.subject_to(V[:, k + 1] == v_next)

    # Phase 1
    if k >= PHASE1_START and k < PHASE1_END:
        add_friction_cone_constraint(opti, U, k, mu, "rear")
        add_friction_cone_constraint(opti, U, k, mu, "front")

        add_contact_constraint(opti, Q, k, q_initial, "rear")
        add_contact_constraint(opti, Q, k, q_initial, "front")

    # Phase 2
    if k >= PHASE2_START and k < PHASE2_END:
        add_friction_cone_constraint(opti, U, k, mu, "rear")
        add_contact_constraint(opti, Q, k, q_initial, "rear")
        add_zero_force_constraint(opti, U, k, "front")

    # Phase 3
    if k >= PHASE3_START and k < PHASE3_END:
        add_zero_force_constraint(opti, U, k, "rear")
        add_zero_force_constraint(opti, U, k, "front")

    # Phase 4
    if k >= PHASE4_START and k < PHASE4_END:
        add_friction_cone_constraint(opti, U, k, mu, "front")
        add_contact_constraint(opti, Q, k, q_final, "front")
        add_zero_force_constraint(opti, U, k, "rear")

    # Phase 5
    if k >= PHASE5_START and k < PHASE5_END:
        add_friction_cone_constraint(opti, U, k, mu, "front")
        add_friction_cone_constraint(opti, U, k, mu, "rear")

        add_contact_constraint(opti, Q, k, q_final, "front")
        add_contact_constraint(opti, Q, k, q_final, "rear")

def is_contact(n):
    return n < int(0.35 * N) or n > int(0.65 * N)


cost = 0
for k in range(N - 1):
    q_angles = Q[3:, k]
    q_angles_default = q_initial[3:]
    q_angles_diff = q_angles - q_angles_default

    for i in range(4):
        angle = q_angles_diff[i]
        opti.subject_to(angle >= -np.pi / 2)
        opti.subject_to(angle <= np.pi / 2)

    cost += 2000 * ca.dot(q_angles_diff, q_angles_diff)
    uext = U[:4, k]
    cost += 2 * ca.dot(uext, uext)
    uext = U[4:, k]
    cost += 500 * ca.dot(uext, uext)

    phi = Q[2, k]

    alpha = k / N

    cost += 1 * (phi - (q_initial[2] + alpha * (q_final[2] - q_initial[2]))) ** 2


opti.minimize(cost)


time = np.linspace(0, N * dt, N)
for i, t in enumerate(time):
    if is_contact(i):
        if i < N - 1:
            opti.set_initial(U[1, i], 0)
            opti.set_initial(U[2, i], 0)
            opti.set_initial(U[3, i], 0)
            opti.set_initial(U[4, i], 0)
    else:
        if i < N - 1:
            opti.set_initial(U[1, i], m * 9.81 / 2)
            opti.set_initial(U[3, i], m * 9.81 / 2)

    alpha = i / N
    opti.set_initial(Q[:, i], q_initial + alpha * (q_final - q_initial))


opti.solver("ipopt")
sol = opti.solve()

U_sol = sol.value(U)
print(U_sol[:4, :])
print(U_sol[:4, :] >= -1e-4)

Q_sol = sol.value(Q)



visualizer = QuadrupedVisualizer(L, leg_length)
visualizer.set_data(Q_sol[:, -1], None, Q_sol[:2, :-1])
plt.show()

def animate(i):
    visualizer.set_data(Q_sol[:, i], None if i == N - 1 else U_sol[:, i], Q_sol[:2, :i])
    return (visualizer.ax,)

anim = animation.FuncAnimation(visualizer.fig, animate, frames=N, interval=50, blit=False)
plt.close()

html = HTML(anim.to_jshtml())
display(html)

if save_gifs:
    anim.save("quadruped_1.gif")



In [ ]:
# Create state vector [x, y, phi, theta1-4]
q_initial = np.array([1.7, 1.2, 0, -1, 1.4, 1, -1.4])
v_initial = np.array([0, 0, 0])

q_final = np.array([-0.3, 1.0, 2 * np.pi, -1, 1.4, 1, -1.4])
v_final = np.array([0, 0, 0])


visualizer = QuadrupedVisualizer(L, leg_length)
visualizer.set_data(q_initial)
plt.show()

q = ca.SX.sym("q", nq)
v = ca.SX.sym("v", nv)
u = ca.SX.sym("u", nu)

dt = 0.02

q_next, v_next = euler_integrate(q, v, u, dt)

get_next_state = ca.Function("get_next_state", [q, v, u], [q_next, v_next])

opti = ca.Opti()

N = 100

Q = opti.variable(nq, N)
V = opti.variable(nv, N)
U = opti.variable(nu, N - 1)


opti.subject_to(Q[:, 0] == q_initial)
opti.subject_to(V[:, 0] == v_initial)

opti.subject_to(Q[:, -1] == q_final)
opti.subject_to(V[:, -1] == v_final)

# Both feet on the ground
PHASE1_START = 0
PHASE1_END = int(0.25 * N)

# One foot on the ground
PHASE2_START = PHASE1_END
PHASE2_END = int(0.35 * N)

# Both feet off the ground
PHASE3_START = PHASE2_END
PHASE3_END = int(0.65 * N)

# One foot on the ground
PHASE4_START = PHASE3_END
PHASE4_END = int(0.68 * N)

# Both feet off the ground
PHASE5_START = PHASE4_END
PHASE5_END = N

def add_friction_cone_constraint(opti, U, k, mu, leg):

    if leg == "front":
        index = [0, 1]
    elif leg == "rear":
        index = [2, 3]
    else:
        raise ValueError("Invalid leg")

    fx = U[index[0], k]
    fy = U[index[1], k]
    opti.subject_to(fx >= -mu * fy)
    opti.subject_to(fx <= mu * fy)

def add_zero_force_constraint(opti, U, k, leg):
    if leg == "front":
        index = [0, 1]
    elif leg == "rear":
        index = [2, 3]
    else:
        raise ValueError("Invalid leg")
    
    fx = U[index[0], k]
    fy = U[index[1], k]
    opti.subject_to(fx == 0)
    opti.subject_to(fy == 0)

def add_contact_constraint(opti, Q, k, q, leg):
    fk_current = forward_kinematics(q)
    fk_symbolic = forward_kinematics(Q[:, k])
    if leg == "front":
        current_front_leg_x, current_front_leg_y = fk_current.front_ankle_x, fk_current.front_ankle_y
        front_leg_x, front_leg_y = fk_symbolic.front_ankle_x, fk_symbolic.front_ankle_y

        opti.subject_to(front_leg_x == current_front_leg_x)
        opti.subject_to(front_leg_y == current_front_leg_y)
    elif leg == "rear":
        current_rear_leg_x, current_rear_leg_y = fk_current.rear_ankle_x, fk_current.rear_ankle_y
        rear_leg_x, rear_leg_y = fk_symbolic.rear_ankle_x, fk_symbolic.rear_ankle_y

        opti.subject_to(rear_leg_x == current_rear_leg_x)
        opti.subject_to(rear_leg_y == current_rear_leg_y)
    else:
        raise ValueError("Invalid leg")

mu = 0.99
for k in range(N - 1):
    q_next, v_next = get_next_state(Q[:, k], V[:, k], U[:, k])
    opti.subject_to(Q[:, k + 1] == q_next)
    opti.subject_to(V[:, k + 1] == v_next)

    # Phase 1
    if k >= PHASE1_START and k < PHASE1_END:
        add_friction_cone_constraint(opti, U, k, mu, "rear")
        add_friction_cone_constraint(opti, U, k, mu, "front")

        add_contact_constraint(opti, Q, k, q_initial, "rear")
        add_contact_constraint(opti, Q, k, q_initial, "front")

    # Phase 2
    if k >= PHASE2_START and k < PHASE2_END:
        add_friction_cone_constraint(opti, U, k, mu, "rear")
        add_contact_constraint(opti, Q, k, q_initial, "rear")
        add_zero_force_constraint(opti, U, k, "front")

    # Phase 3
    if k >= PHASE3_START and k < PHASE3_END:
        add_zero_force_constraint(opti, U, k, "rear")
        add_zero_force_constraint(opti, U, k, "front")

    # Phase 4
    if k >= PHASE4_START and k < PHASE4_END:
        add_friction_cone_constraint(opti, U, k, mu, "front")
        add_contact_constraint(opti, Q, k, q_final, "front")
        add_zero_force_constraint(opti, U, k, "rear")

    # Phase 5
    if k >= PHASE5_START and k < PHASE5_END:
        add_friction_cone_constraint(opti, U, k, mu, "front")
        add_friction_cone_constraint(opti, U, k, mu, "rear")

        add_contact_constraint(opti, Q, k, q_final, "front")
        add_contact_constraint(opti, Q, k, q_final, "rear")

def is_contact(n):
    return n < int(0.35 * N) or n > int(0.65 * N)


cost = 0
for k in range(N - 1):
    q_angles = Q[3:, k]
    q_angles_default = q_initial[3:]
    q_angles_diff = q_angles - q_angles_default

    for i in range(4):
        angle = q_angles_diff[i]
        opti.subject_to(angle >= -np.pi / 2)
        opti.subject_to(angle <= np.pi / 2)

    cost += 2000 * ca.dot(q_angles_diff, q_angles_diff)
    uext = U[:4, k]
    cost += 2 * ca.dot(uext, uext)
    uext = U[4:, k]
    cost += 500 * ca.dot(uext, uext)

    phi = Q[2, k]

    alpha = k / N

    cost += 1 * (phi - (q_initial[2] + alpha * (q_final[2] - q_initial[2]))) ** 2


opti.minimize(cost)


time = np.linspace(0, N * dt, N)
for i, t in enumerate(time):
    if is_contact(i):
        if i < N - 1:
            opti.set_initial(U[1, i], 0)
            opti.set_initial(U[2, i], 0)
            opti.set_initial(U[3, i], 0)
            opti.set_initial(U[4, i], 0)
    else:
        if i < N - 1:
            opti.set_initial(U[1, i], m * 9.81 / 2)
            opti.set_initial(U[3, i], m * 9.81 / 2)

    alpha = i / N
    opti.set_initial(Q[:, i], q_initial + alpha * (q_final - q_initial))


opti.solver("ipopt")
sol = opti.solve()

U_sol = sol.value(U)
print(U_sol[:4, :])
print(U_sol[:4, :] >= -1e-4)

Q_sol = sol.value(Q)



visualizer = QuadrupedVisualizer(L, leg_length)
visualizer.set_data(Q_sol[:, -1], None, Q_sol[:2, :-1])
plt.show()

def animate(i):
    visualizer.set_data(Q_sol[:, i], None if i == N - 1 else U_sol[:, i], Q_sol[:2, :i])
    return (visualizer.ax,)

anim = animation.FuncAnimation(visualizer.fig, animate, frames=N, interval=50, blit=False)
plt.close()

html = HTML(anim.to_jshtml())
display(html)


if save_gifs:
    anim.save("quadruped_2.gif")

In [ ]:
# Create state vector [x, y, phi, theta1-4]
q_initial = np.array([1.7, 1.2, 0, -1, 1.4, 1, -1.4])
v_initial = np.array([0, 0, 0])

q_final = np.array([-0.4, 1.0, 0, -1, 1.4, 1, -1.4])
v_final = np.array([0, 0, 0])


visualizer = QuadrupedVisualizer(L, leg_length)
visualizer.set_data(q_initial)
plt.show()

q = ca.SX.sym("q", nq)
v = ca.SX.sym("v", nv)
u = ca.SX.sym("u", nu)

dt = 0.02

q_next, v_next = euler_integrate(q, v, u, dt)

get_next_state = ca.Function("get_next_state", [q, v, u], [q_next, v_next])

opti = ca.Opti()

N = 100

Q = opti.variable(nq, N)
V = opti.variable(nv, N)
U = opti.variable(nu, N - 1)


opti.subject_to(Q[:, 0] == q_initial)
opti.subject_to(V[:, 0] == v_initial)

opti.subject_to(Q[:, -1] == q_final)
opti.subject_to(V[:, -1] == v_final)

# Both feet on the ground
PHASE1_START = 0
PHASE1_END = int(0.35 * N)

# One foot on the ground
PHASE2_START = PHASE1_END
PHASE2_END = int(0.65 * N)

# Both feet off the ground
PHASE3_START = PHASE2_END
PHASE3_END = N

def add_friction_cone_constraint(opti, U, k, mu, leg):

    if leg == "front":
        index = [0, 1]
    elif leg == "rear":
        index = [2, 3]
    else:
        raise ValueError("Invalid leg")

    fx = U[index[0], k]
    fy = U[index[1], k]
    opti.subject_to(fx >= -mu * fy)
    opti.subject_to(fx <= mu * fy)

def add_zero_force_constraint(opti, U, k, leg):
    if leg == "front":
        index = [0, 1]
    elif leg == "rear":
        index = [2, 3]
    else:
        raise ValueError("Invalid leg")
    
    fx = U[index[0], k]
    fy = U[index[1], k]
    opti.subject_to(fx == 0)
    opti.subject_to(fy == 0)

def add_contact_constraint(opti, Q, k, q, leg):
    fk_current = forward_kinematics(q)
    fk_symbolic = forward_kinematics(Q[:, k])
    if leg == "front":
        current_front_leg_x, current_front_leg_y = fk_current.front_ankle_x, fk_current.front_ankle_y
        front_leg_x, front_leg_y = fk_symbolic.front_ankle_x, fk_symbolic.front_ankle_y

        opti.subject_to(front_leg_x == current_front_leg_x)
        opti.subject_to(front_leg_y == current_front_leg_y)
    elif leg == "rear":
        current_rear_leg_x, current_rear_leg_y = fk_current.rear_ankle_x, fk_current.rear_ankle_y
        rear_leg_x, rear_leg_y = fk_symbolic.rear_ankle_x, fk_symbolic.rear_ankle_y

        opti.subject_to(rear_leg_x == current_rear_leg_x)
        opti.subject_to(rear_leg_y == current_rear_leg_y)
    else:
        raise ValueError("Invalid leg")

mu = 0.99
for k in range(N - 1):
    q_next, v_next = get_next_state(Q[:, k], V[:, k], U[:, k])
    opti.subject_to(Q[:, k + 1] == q_next)
    opti.subject_to(V[:, k + 1] == v_next)

    # Phase 1
    if k >= PHASE1_START and k < PHASE1_END:
        add_friction_cone_constraint(opti, U, k, mu, "rear")
        add_friction_cone_constraint(opti, U, k, mu, "front")

        add_contact_constraint(opti, Q, k, q_initial, "rear")
        add_contact_constraint(opti, Q, k, q_initial, "front")

    # Phase 3
    if k >= PHASE2_START and k < PHASE2_END:
        add_zero_force_constraint(opti, U, k, "rear")
        add_zero_force_constraint(opti, U, k, "front")

    # Phase 5
    if k >= PHASE3_START and k < PHASE3_END:
        add_friction_cone_constraint(opti, U, k, mu, "front")
        add_friction_cone_constraint(opti, U, k, mu, "rear")

        add_contact_constraint(opti, Q, k, q_final, "front")
        add_contact_constraint(opti, Q, k, q_final, "rear")

def is_contact(n):
    return n < int(0.35 * N) or n > int(0.65 * N)


cost = 0
for k in range(N - 1):
    q_angles = Q[3:, k]
    q_angles_default = q_initial[3:]
    q_angles_diff = q_angles - q_angles_default

    for i in range(4):
        angle = q_angles_diff[i]
        opti.subject_to(angle >= -np.pi / 2)
        opti.subject_to(angle <= np.pi / 2)

    cost += 2000 * ca.dot(q_angles_diff, q_angles_diff)
    uext = U[:4, k]
    cost += 2 * ca.dot(uext, uext)
    uext = U[4:, k]
    cost += 500 * ca.dot(uext, uext)

    phi = Q[2, k]

    alpha = k / N

    cost += 1 * (phi - (q_initial[2] + alpha * (q_final[2] - q_initial[2]))) ** 2


opti.minimize(cost)


time = np.linspace(0, N * dt, N)
for i, t in enumerate(time):
    if is_contact(i):
        if i < N - 1:
            opti.set_initial(U[1, i], 0)
            opti.set_initial(U[2, i], 0)
            opti.set_initial(U[3, i], 0)
            opti.set_initial(U[4, i], 0)
    else:
        if i < N - 1:
            opti.set_initial(U[1, i], m * 9.81 / 2)
            opti.set_initial(U[3, i], m * 9.81 / 2)

    alpha = i / N
    opti.set_initial(Q[:, i], q_initial + alpha * (q_final - q_initial))


opti.solver("ipopt")
sol = opti.solve()

U_sol = sol.value(U)
print(U_sol[:4, :])
print(U_sol[:4, :] >= -1e-4)

Q_sol = sol.value(Q)



visualizer = QuadrupedVisualizer(L, leg_length)
visualizer.set_data(Q_sol[:, -1], None, Q_sol[:2, :-1])
plt.show()

def animate(i):
    visualizer.set_data(Q_sol[:, i], None if i == N - 1 else U_sol[:, i], Q_sol[:2, :i])
    return (visualizer.ax,)

anim = animation.FuncAnimation(visualizer.fig, animate, frames=N, interval=50, blit=False)
plt.close()

html = HTML(anim.to_jshtml())
display(html)


if save_gifs:
    anim.save("quadruped_3.gif")

In [ ]:
# Create state vector [x, y, phi, theta1-4]
q_initial = np.array([1.7, 1.2, 0, -1, 1.4, 1, -1.4])
v_initial = np.array([0, 0, 0])

q_final = np.array([-0.4, 1.0, 0, -1, 1.4, 1, -1.4])
v_final = np.array([0, 0, 0])


visualizer = QuadrupedVisualizer(L, leg_length)
visualizer.set_data(q_initial)
plt.show()

q = ca.SX.sym("q", nq)
v = ca.SX.sym("v", nv)
u = ca.SX.sym("u", nu)

dt = 0.02

q_next, v_next = euler_integrate(q, v, u, dt)

get_next_state = ca.Function("get_next_state", [q, v, u], [q_next, v_next])

opti = ca.Opti()

N = 200

Q = opti.variable(nq, N)
V = opti.variable(nv, N)
U = opti.variable(nu, N - 1)


opti.subject_to(Q[:, 0] == q_initial)
opti.subject_to(V[:, 0] == v_initial)

opti.subject_to(Q[:, -1] == q_initial)
opti.subject_to(V[:, -1] == v_initial)

# Both feet on the ground
PHASE1_START = 0
PHASE1_END = int(0.35 * N / 2)

# One foot on the ground
PHASE2_START = PHASE1_END
PHASE2_END = int(0.65 * N / 2)

# Both feet on the ground
PHASE3_START = PHASE2_END
PHASE3_END = int(N / 2)

PHASE4_START = PHASE3_END
PHASE4_END = int(0.35 * N / 2 + N / 2)

PHASE5_START = PHASE4_END
PHASE5_END = int(0.65 * N / 2 + N / 2)

PHASE6_START = PHASE5_END
PHASE6_END = N

opti.subject_to(Q[:, PHASE3_END] == q_final)
opti.subject_to(V[:, PHASE3_END] == v_final)


def add_friction_cone_constraint(opti, U, k, mu, leg):

    if leg == "front":
        index = [0, 1]
    elif leg == "rear":
        index = [2, 3]
    else:
        raise ValueError("Invalid leg")

    fx = U[index[0], k]
    fy = U[index[1], k]
    opti.subject_to(fx >= -mu * fy)
    opti.subject_to(fx <= mu * fy)

def add_zero_force_constraint(opti, U, k, leg):
    if leg == "front":
        index = [0, 1]
    elif leg == "rear":
        index = [2, 3]
    else:
        raise ValueError("Invalid leg")
    
    fx = U[index[0], k]
    fy = U[index[1], k]
    opti.subject_to(fx == 0)
    opti.subject_to(fy == 0)

def add_contact_constraint(opti, Q, k, q, leg):
    fk_current = forward_kinematics(q)
    fk_symbolic = forward_kinematics(Q[:, k])
    if leg == "front":
        current_front_leg_x, current_front_leg_y = fk_current.front_ankle_x, fk_current.front_ankle_y
        front_leg_x, front_leg_y = fk_symbolic.front_ankle_x, fk_symbolic.front_ankle_y

        opti.subject_to(front_leg_x == current_front_leg_x)
        opti.subject_to(front_leg_y == current_front_leg_y)
    elif leg == "rear":
        current_rear_leg_x, current_rear_leg_y = fk_current.rear_ankle_x, fk_current.rear_ankle_y
        rear_leg_x, rear_leg_y = fk_symbolic.rear_ankle_x, fk_symbolic.rear_ankle_y

        opti.subject_to(rear_leg_x == current_rear_leg_x)
        opti.subject_to(rear_leg_y == current_rear_leg_y)
    else:
        raise ValueError("Invalid leg")

mu = 0.99
for k in range(N - 1):
    q_next, v_next = get_next_state(Q[:, k], V[:, k], U[:, k])
    opti.subject_to(Q[:, k + 1] == q_next)
    opti.subject_to(V[:, k + 1] == v_next)

    # Phase 1
    if k >= PHASE1_START and k < PHASE1_END:
        add_friction_cone_constraint(opti, U, k, mu, "rear")
        add_friction_cone_constraint(opti, U, k, mu, "front")

        add_contact_constraint(opti, Q, k, q_initial, "rear")
        add_contact_constraint(opti, Q, k, q_initial, "front")

    # Phase 3
    if k >= PHASE2_START and k < PHASE2_END:
        add_zero_force_constraint(opti, U, k, "rear")
        add_zero_force_constraint(opti, U, k, "front")

    # Phase 5
    if k >= PHASE3_START and k < PHASE3_END:
        add_friction_cone_constraint(opti, U, k, mu, "front")
        add_friction_cone_constraint(opti, U, k, mu, "rear")

        add_contact_constraint(opti, Q, k, q_final, "front")
        add_contact_constraint(opti, Q, k, q_final, "rear")

    if k >= PHASE4_START and k < PHASE4_END:
        add_friction_cone_constraint(opti, U, k, mu, "front")
        add_friction_cone_constraint(opti, U, k, mu, "rear")

        add_contact_constraint(opti, Q, k, q_final, "front")
        add_contact_constraint(opti, Q, k, q_final, "rear")


    if k >= PHASE5_START and k < PHASE5_END:
        add_zero_force_constraint(opti, U, k, "front")
        add_zero_force_constraint(opti, U, k, "rear")

    if k >= PHASE6_START and k < PHASE6_END:
        add_friction_cone_constraint(opti, U, k, mu, "front")
        add_friction_cone_constraint(opti, U, k, mu, "rear")

        add_contact_constraint(opti, Q, k, q_initial, "front")
        add_contact_constraint(opti, Q, k, q_initial, "rear")


cost = 0
for k in range(N - 1):
    q_angles = Q[3:, k]
    q_angles_default = q_initial[3:]
    q_angles_diff = q_angles - q_angles_default

    for i in range(4):
        angle = q_angles_diff[i]
        opti.subject_to(angle >= -np.pi / 2)
        opti.subject_to(angle <= np.pi / 2)

    cost += 2000 * ca.dot(q_angles_diff, q_angles_diff)
    uext = U[:4, k]
    cost += 2 * ca.dot(uext, uext)
    uext = U[4:, k]
    cost += 500 * ca.dot(uext, uext)

    phi = Q[2, k]

    alpha = k / N

    cost += 1 * (phi - (q_initial[2] + alpha * (q_final[2] - q_initial[2]))) ** 2


opti.minimize(cost)


time = np.linspace(0, N * dt, N)
for i, t in enumerate(time):
    if i < N-1: 
        opti.set_initial(U[0, i], 0)
        opti.set_initial(U[1, i], m * 9.81 / 2)
        opti.set_initial(U[2, i], 0)
        opti.set_initial(U[3, i], m * 9.81 / 2)

    if i < PHASE3_END:
        alpha = i / PHASE3_END
        opti.set_initial(Q[:, i], q_initial + alpha * (q_final - q_initial))
    else:
        alpha = (i - PHASE3_END) / (N - PHASE3_END)
        opti.set_initial(Q[:, i], q_final + alpha * (q_initial - q_final))


opti.solver("ipopt")
sol = opti.solve()

U_sol = sol.value(U)
print(U_sol[:4, :])
print(U_sol[:4, :] >= -1e-4)

Q_sol = sol.value(Q)



visualizer = QuadrupedVisualizer(L, leg_length)
visualizer.set_data(Q_sol[:, -1], None, Q_sol[:2, :-1])
plt.show()

def animate(i):
    visualizer.set_data(Q_sol[:, i], None if i == N - 1 else U_sol[:, i], Q_sol[:2, :i])
    return (visualizer.ax,)

anim = animation.FuncAnimation(visualizer.fig, animate, frames=N, interval=50, blit=False)
plt.close()

html = HTML(anim.to_jshtml())
display(html)



if save_gifs:
    anim.save("quadruped_4.gif")

In [ ]:
# Create state vector [x, y, phi, theta1-4]
q_initial = np.array([1.7, 1.2, 0, -1, 1.4, 1, -1.4])
v_initial = np.array([0, 0, 0])

q_mid = np.array([-0.4, 1.0, 0, -1, 1.4, 1, -1.4])
v_mid = np.array([0, 0, 0])

q_final = np.array([1.7, 1.2, -2*np.pi, -1, 1.4, 1, -1.4])
v_final = np.array([0, 0, 0])

visualizer = QuadrupedVisualizer(L, leg_length)
visualizer.set_data(q_initial)
plt.show()

q = ca.SX.sym("q", nq)
v = ca.SX.sym("v", nv)
u = ca.SX.sym("u", nu)

dt = 0.02

q_next, v_next = euler_integrate(q, v, u, dt)

get_next_state = ca.Function("get_next_state", [q, v, u], [q_next, v_next])

opti = ca.Opti()

N = 200

Q = opti.variable(nq, N)
V = opti.variable(nv, N)
U = opti.variable(nu, N - 1)


opti.subject_to(Q[:, 0] == q_initial)
opti.subject_to(V[:, 0] == v_initial)

opti.subject_to(Q[:, -1] == q_final)
opti.subject_to(V[:, -1] == v_final)

# Both feet on the ground
PHASE1_START = 0
PHASE1_END = int(0.35 * N / 2)

# One foot on the ground
PHASE2_START = PHASE1_END
PHASE2_END = int(0.65 * N / 2)

# Both feet on the ground
PHASE3_START = PHASE2_END
PHASE3_END = int(N / 2)

PHASE4_START = PHASE3_END
PHASE4_END = int(0.29 * N / 2 + N / 2)

PHASE5_START = PHASE4_END
PHASE5_END = int(0.35 * N / 2 + N / 2)

PHASE6_START = PHASE5_END
PHASE6_END = int(0.65 * N / 2 + N / 2)

PHASE7_START = PHASE6_END
PHASE7_END = int(0.68 * N / 2 + N / 2)

PHASE8_START = PHASE7_END
PHASE8_END = N

opti.subject_to(Q[:, PHASE3_END] == q_mid)
opti.subject_to(V[:, PHASE3_END] == v_mid)


def add_friction_cone_constraint(opti, U, k, mu, leg):

    if leg == "front":
        index = [0, 1]
    elif leg == "rear":
        index = [2, 3]
    else:
        raise ValueError("Invalid leg")

    fx = U[index[0], k]
    fy = U[index[1], k]
    opti.subject_to(fx >= -mu * fy)
    opti.subject_to(fx <= mu * fy)

def add_zero_force_constraint(opti, U, k, leg):
    if leg == "front":
        index = [0, 1]
    elif leg == "rear":
        index = [2, 3]
    else:
        raise ValueError("Invalid leg")
    
    fx = U[index[0], k]
    fy = U[index[1], k]
    opti.subject_to(fx == 0)
    opti.subject_to(fy == 0)

def add_contact_constraint(opti, Q, k, q, leg):
    fk_current = forward_kinematics(q)
    fk_symbolic = forward_kinematics(Q[:, k])
    if leg == "front":
        current_front_leg_x, current_front_leg_y = fk_current.front_ankle_x, fk_current.front_ankle_y
        front_leg_x, front_leg_y = fk_symbolic.front_ankle_x, fk_symbolic.front_ankle_y

        opti.subject_to(front_leg_x == current_front_leg_x)
        opti.subject_to(front_leg_y == current_front_leg_y)
    elif leg == "rear":
        current_rear_leg_x, current_rear_leg_y = fk_current.rear_ankle_x, fk_current.rear_ankle_y
        rear_leg_x, rear_leg_y = fk_symbolic.rear_ankle_x, fk_symbolic.rear_ankle_y

        opti.subject_to(rear_leg_x == current_rear_leg_x)
        opti.subject_to(rear_leg_y == current_rear_leg_y)
    else:
        raise ValueError("Invalid leg")

mu = 0.99
for k in range(N - 1):
    q_next, v_next = get_next_state(Q[:, k], V[:, k], U[:, k])
    opti.subject_to(Q[:, k + 1] == q_next)
    opti.subject_to(V[:, k + 1] == v_next)

    # Phase 1
    if k >= PHASE1_START and k < PHASE1_END:
        add_friction_cone_constraint(opti, U, k, mu, "rear")
        add_friction_cone_constraint(opti, U, k, mu, "front")

        add_contact_constraint(opti, Q, k, q_initial, "rear")
        add_contact_constraint(opti, Q, k, q_initial, "front")

    # Phase 3
    if k >= PHASE2_START and k < PHASE2_END:
        add_zero_force_constraint(opti, U, k, "rear")
        add_zero_force_constraint(opti, U, k, "front")

    # Phase 5
    if k >= PHASE3_START and k < PHASE3_END:
        add_friction_cone_constraint(opti, U, k, mu, "front")
        add_friction_cone_constraint(opti, U, k, mu, "rear")

        add_contact_constraint(opti, Q, k, q_mid, "front")
        add_contact_constraint(opti, Q, k, q_mid, "rear")

    if k >= PHASE4_START and k < PHASE4_END:
        add_friction_cone_constraint(opti, U, k, mu, "front")
        add_friction_cone_constraint(opti, U, k, mu, "rear")

        add_contact_constraint(opti, Q, k, q_mid, "front")
        add_contact_constraint(opti, Q, k, q_mid, "rear")


    if k >= PHASE5_START and k < PHASE5_END:
        add_friction_cone_constraint(opti, U, k, mu, "front")
        add_contact_constraint(opti, Q, k, q_mid, "front")

        add_zero_force_constraint(opti, U, k, "rear")

    if k >= PHASE6_START and k < PHASE6_END:
        add_zero_force_constraint(opti, U, k, "front")
        add_zero_force_constraint(opti, U, k, "rear")

    if k >= PHASE7_START and k < PHASE7_END:
        add_friction_cone_constraint(opti, U, k, mu, "rear")
        add_contact_constraint(opti, Q, k, q_final, "rear")

        add_zero_force_constraint(opti, U, k, "front")

    if k >= PHASE8_START and k < PHASE8_END:
        add_friction_cone_constraint(opti, U, k, mu, "front")
        add_contact_constraint(opti, Q, k, q_final, "front")
        add_friction_cone_constraint(opti, U, k, mu, "rear")
        add_contact_constraint(opti, Q, k, q_final, "rear")


cost = 0
for k in range(N - 1):
    q_angles = Q[3:, k]
    q_angles_default = q_initial[3:]
    q_angles_diff = q_angles - q_angles_default

    for i in range(4):
        angle = q_angles_diff[i]
        opti.subject_to(angle >= -np.pi / 2)
        opti.subject_to(angle <= np.pi / 2)

    cost += 2000 * ca.dot(q_angles_diff, q_angles_diff)
    uext = U[:4, k]
    cost += 2 * ca.dot(uext, uext)
    uext = U[4:, k]
    cost += 500 * ca.dot(uext, uext)

    phi = Q[2, k]

    alpha = k / N

    if k < PHASE3_END:
        alpha = k / PHASE3_END
        cost += 1 * (phi - (q_initial[2] + alpha * (q_mid[2] - q_initial[2]))) ** 2
    else:
        alpha = (k - PHASE3_END) / (N - PHASE3_END)
        cost += 1 * (phi - (q_mid[2] + alpha * (q_final[2] - q_mid[2]))) ** 2


opti.minimize(cost)


time = np.linspace(0, N * dt, N)
for i, t in enumerate(time):
    if i < N-1: 
        opti.set_initial(U[0, i], 0)
        opti.set_initial(U[1, i], m * 9.81 / 2)
        opti.set_initial(U[2, i], 0)
        opti.set_initial(U[3, i], m * 9.81 / 2)

    if i < PHASE3_END:
        alpha = i / PHASE3_END
        opti.set_initial(Q[:, i], q_initial + alpha * (q_mid - q_initial))
    else:
        alpha = (i - PHASE3_END) / (N - PHASE3_END)
        opti.set_initial(Q[:, i], q_mid + alpha * (q_final - q_mid))


opti.solver("ipopt")
sol = opti.solve()

U_sol = sol.value(U)
print(U_sol[:4, :])
print(U_sol[:4, :] >= -1e-4)

Q_sol = sol.value(Q)



visualizer = QuadrupedVisualizer(L, leg_length)
visualizer.set_data(Q_sol[:, -1], None, Q_sol[:2, :-1])
plt.show()

def animate(i):
    visualizer.set_data(Q_sol[:, i], None if i == N - 1 else U_sol[:, i], Q_sol[:2, :i])
    return (visualizer.ax,)

anim = animation.FuncAnimation(visualizer.fig, animate, frames=N, interval=50, blit=False)
plt.close()

html = HTML(anim.to_jshtml())
display(html)


if save_gifs:
    anim.save("quadruped_5.gif")